In [1]:
from math import sqrt
from numpy import concatenate
import numpy as np
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout
from tensorflow.keras.models import load_model

In [2]:
np.set_printoptions(suppress = True)

#convert series to supervised learning
def series_to_supervised(data, n_in = 1, n_out = 1, dropnan = True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()

    #input sequence (t-n,...t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1,i)) for j in range(n_vars)]
    
    #forcast sequence(t,t+1,...t+n)
    for i in range(0,n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    
    #put it all together
    agg = concat(cols, axis = 1)
    agg.columns = names
    
    #drop rows with NaN values
    if dropnan:
        agg.dropna(inplace = True)
    
    return agg



In [3]:
#load dataset
col_list = ["H2S", "PH", "Temp"]
dataset = read_csv('DATASET_2019.csv', usecols=col_list)
dataset = dataset[["H2S", "PH", "Temp"]]
values = dataset.values

In [4]:
#ensure all data is float
values = values.astype('float32')

In [5]:
#normalize features
scaler = MinMaxScaler(feature_range=(0,1))
scaled =  scaler.fit_transform(values)

In [6]:
#specific the number of lag hours
n_hours = 60
n_features = 3
n_obs = n_hours * n_features

In [51]:
#frame as supervised learning
#reframed = series_to_supervised(values, n_hours, 1)
reframed = series_to_supervised(scaled, n_hours, 1)
reframed.drop(reframed.columns[[n_obs,(n_obs+1)]], axis = 1, inplace = True)
print(reframed.head())

     var1(t-60)  var2(t-60)  var3(t-60)  var1(t-59)  var2(t-59)  var3(t-59)  \
271    0.000000    0.882353     0.00000    0.000000    0.823530     0.03125   
272    0.000000    0.823530     0.03125    0.333333    0.647059     0.06250   
273    0.333333    0.647059     0.06250    0.333333    0.647059     0.09375   
274    0.333333    0.647059     0.09375    0.000000    0.823530     0.06875   
275    0.000000    0.823530     0.06875    0.000000    0.823530     0.06875   

     var1(t-58)  var2(t-58)  var3(t-58)  var1(t-57)  ...  var1(t-3)  \
271    0.333333    0.647059     0.06250    0.333333  ...   0.033333   
272    0.333333    0.647059     0.09375    0.000000  ...   0.033333   
273    0.000000    0.823530     0.06875    0.000000  ...   0.033333   
274    0.000000    0.823530     0.06875    0.000000  ...   0.033333   
275    0.000000    0.823530     0.08750    0.000000  ...   0.033333   

     var2(t-3)  var3(t-3)  var1(t-2)  var2(t-2)  var3(t-2)  var1(t-1)  \
271   0.882353    0.48125

In [52]:
#split into train and test sets
values = reframed.values
n_train_hours = int(len(dataset)*0.7)
train = values[:n_train_hours, :]
test = values[n_train_hours:,:]

In [53]:
#split into input and outputs
train_X, train_y = train[:,:n_obs], train[:,-1]
test_X, test_y = test[:,:n_obs], test[:,-1]
print(train_X.shape, len(train_X), train_y.shape)

(1533, 180) 1533 (1533,)


In [65]:
#reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(1533, 60, 3) (1533,) (294, 60, 3) (294, 1)


In [71]:
#make a prediction
model = Sequential()
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_obs))
# yha = yhat.reshape(yhat.shape[0], yhat.shape[1])
yhat.shape

10/10 [==============================] - 0s 1ms/step


(294, 180)

In [ ]:
# invert scaling for forecast
inv_yhat = concatenate((test_X[:, [0,1]], yhat), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:, 2]

In [62]:
#invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_X[:,[0,1]], test_y), axis = 1)
inv_y = scaler.inverse_transform(inv_y)
# inv_y = inv_y[:,2]
test_X[:,[0,1]]

array([[0.16666667, 0.8235297 ],
       [0.        , 0.8823533 ],
       [0.        , 0.9411764 ],
       [0.        , 0.8823533 ],
       [0.        , 1.0000005 ],
       [0.        , 0.8235297 ],
       [0.03333334, 0.8235297 ],
       [0.        , 0.9411764 ],
       [0.        , 0.8823533 ],
       [0.03333334, 0.8235297 ],
       [0.03333334, 0.8823533 ],
       [0.06666668, 0.76470613],
       [0.06666668, 0.76470613],
       [0.03333334, 0.76470613],
       [0.03333334, 0.76470613],
       [0.06666668, 0.70588255],
       [0.03333334, 0.70588255],
       [0.10000001, 0.70588255],
       [0.06666668, 0.76470613],
       [0.03333334, 0.76470613],
       [0.        , 0.76470613],
       [0.        , 0.76470613],
       [0.        , 0.76470613],
       [0.        , 0.76470613],
       [0.        , 0.76470613],
       [0.        , 0.76470613],
       [0.        , 0.76470613],
       [0.        , 0.76470613],
       [0.        , 0.76470613],
       [0.        , 0.76470613],
       [0.

In [ ]:
# Visualising the results
pyplot.figure(figsize=(14,5))
pyplot.plot(inv_y, color = 'red', label = 'actual')
pyplot.plot(inv_yhat, color = 'blue', label = 'predict by multi params')
pyplot.xlabel('Sampling Point', fontsize=15)
pyplot.ylabel('Temperature', fontsize=15)
pyplot.legend()
pyplot.savefig("temperature_multi_params.svg", format = 'svg', dpi=300)
pyplot.show()

In [ ]:
#calculaate RSME
mse = mean_squared_error(inv_y, inv_yhat)
rmse = sqrt(mse)
print('Test MSE: %.3f' % mse)
print('Test RMSE: %.3f' % rmse)
lose_model = abs(inv_y - inv_yhat)
lose_model = lose_model.reshape(-1,1)
print(lose_model.shape)

mean = np.mean(lose_model)
min = np.amin(lose_model)
max = np.amax(lose_model)
range = np.ptp(lose_model)
varience = np.var(lose_model)
sd = np.std(lose_model)
print("Measures of Dispersion")
print("Mean = ",  mean)
print("Minimum = ", min)
print("Maximum =  ", max)
print("Range = ", range)
print("Varience = ", varience)
print("Standard Deviation = ", sd)